In [1]:
# Initial Imports
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from pathlib import Path
import regex as re
import nltk
from datetime import date
import twint
import nest_asyncio
nest_asyncio.apply()
nltk.download('stopwords')

%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\julia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Initialize set of inputs
# List of Stocks
stocks = ['microsoft','sony','pfizer','regeneron','moderna']

# Set up Date range
import datetime
from dateutil.relativedelta import relativedelta

today = datetime.date.today()
end_time = today.strftime('%Y-%m-%d %H:%M:%S')
start_time = today - relativedelta(years=1)
start_time = start_time.strftime('%Y-%m-%d %H:%M:%S')
print(start_time)

2020-01-21 00:00:00


In [3]:
# Define Empty DataFrames
stock_df = pd.DataFrame()
df = pd.DataFrame()

In [4]:
# Define data_crawlers function
def data_crawlers(stock):
    c = twint.Config()
    c.Search = stock
    c.Hide_output = True
    c.Limit = 1000 
    #c.Store_csv = True
    c.Since = start_time
    c.Until = end_time
    #c.Output = f"{stock}.csv"
    c.Pandas = True
    c.Pandas_au = True
    twint.run.Search(c)
    df = twint.storage.panda.Tweets_df
    df.index=idx_microsoft=pd.MultiIndex.from_tuples([tuple([stock, i]) for i in range(len(df.index))])
    return df

In [5]:
# Test function
df_test=data_crawlers(stocks[0])
df_test.tail()

id      conversation_id    created_at  \
microsoft 1090  1351653168980107271  1351653168980107271  1.611094e+12   
          1091  1351653160817975296  1351653160817975296  1.611094e+12   
          1092  1351653138953068544  1351651325788913664  1.611094e+12   
          1093  1351653108364017665  1351653108364017665  1.611094e+12   
          1094  1351653079230390272  1350190230133997572  1.611094e+12   

                               date timezone place  \
microsoft 1090  2021-01-19 17:10:02    -0500         
          1091  2021-01-19 17:10:00    -0500         
          1092  2021-01-19 17:09:55    -0500         
          1093  2021-01-19 17:09:48    -0500         
          1094  2021-01-19 17:09:41    -0500         

                                                            tweet language  \
microsoft 1090  Cruise, GM partner with Microsoft to commercia...       en   
          1091  eu to rindo muito com a história da vez que a ...       pt   
          1092  @enertopiacorp @GM @Microsoft As usual nothing...       en   
          1093  @MartinSLewis Hi I purchased a faulty XBox X S...       en   
          1094  @microsoftnl Ik vraag me nog steeds af waarom ...       nl   

               hashtags cashtags  ...  geo source user_rt_id user_rt  \
microsoft 1090       []       []  ...                                  
          1091       []       []  ...                                  
          1092       []       []  ...                                  
          1093       []       []  ...                                  
          1094       []       []  ...                                  

                retweet_id                                           reply_to  \
microsoft 1090                                                             []   
          1091                                                             []   
          1092              [{'screen_name': 'enertopiacorp', 'name': 'Ene...   
          1093                                                             []   
          1094              [{'screen_name': 'microsoftnl', 'name': 'Micro...   

               retweet_date translate trans_src  trans_dest  
microsoft 1090                                               
          1091                                               
          1092                                               
          1093                                               
          1094                                               

[5 rows x 38 columns]

In [6]:
df_test['date'] = [x[:10] for x in df_test['date']]

In [7]:
df = df_test.copy()
df.reset_index(inplace=True)

In [8]:
df.head()

,level_0,level_1,id,conversation_id,created_at,date,timezone,place,tweet,language,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,microsoft,0,1351680836580601857,1351600853459857410,1.611101e+12,2021-01-19,-0500,,@grimshaw_leon @Treyarch @Microsoft I’m not un...,en,...,,,,,,"[{'screen_name': 'grimshaw_leon', 'name': 'leo...",,,,
1,microsoft,1,1351680824828166144,1351680824828166144,1.611101e+12,2021-01-19,-0500,,I won 4 achievements in Microsoft Sudoku (UWP)...,en,...,,,,,,[],,,,
2,microsoft,2,1351680820927467521,1351680820927467521,1.611101e+12,2021-01-19,-0500,,I won the Flipping Out achievement in Microsof...,en,...,,,,,,[],,,,
3,microsoft,3,1351680820252033024,1351680820252033024,1.611101e+12,2021-01-19,-0500,,Be it @Microsoft or @Google CEO but... फिर भी...,hi,...,,,,,,[],,,,
4,microsoft,4,1351680814942195713,1351680814942195713,1.611101e+12,2021-01-19,-0500,,I won the Mummy’s Boy achievement in Microsoft...,en,...,,,,,,[],,,,


In [9]:
df = df.drop(columns='level_1')
df.rename(columns={'level_0':'Stock'})

,Stock,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,microsoft,1351680836580601857,1351600853459857410,1.611101e+12,2021-01-19,-0500,,@grimshaw_leon @Treyarch @Microsoft I’m not un...,en,[],...,,,,,,"[{'screen_name': 'grimshaw_leon', 'name': 'leo...",,,,
1,microsoft,1351680824828166144,1351680824828166144,1.611101e+12,2021-01-19,-0500,,I won 4 achievements in Microsoft Sudoku (UWP)...,en,[trueachievements],...,,,,,,[],,,,
2,microsoft,1351680820927467521,1351680820927467521,1.611101e+12,2021-01-19,-0500,,I won the Flipping Out achievement in Microsof...,en,[trueachievements],...,,,,,,[],,,,
3,microsoft,1351680820252033024,1351680820252033024,1.611101e+12,2021-01-19,-0500,,Be it @Microsoft or @Google CEO but... फिर भी...,hi,[indvsaus],...,,,,,,[],,,,
4,microsoft,1351680814942195713,1351680814942195713,1.611101e+12,2021-01-19,-0500,,I won the Mummy’s Boy achievement in Microsoft...,en,[trueachievements],...,,,,,,[],,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,microsoft,1351653168980107271,1351653168980107271,1.611094e+12,2021-01-19,-0500,,"Cruise, GM partner with Microsoft to commercia...",en,[],...,,,,,,[],,,,
1091,microsoft,1351653160817975296,1351653160817975296,1.611094e+12,2021-01-19,-0500,,eu to rindo muito com a história da vez que a ...,pt,[],...,,,,,,[],,,,
1092,microsoft,1351653138953068544,1351651325788913664,1.611094e+12,2021-01-19,-0500,,@enertopiacorp @GM @Microsoft As usual nothing...,en,[],...,,,,,,"[{'screen_name': 'enertopiacorp', 'name': 'Ene...",,,,
1093,microsoft,1351653108364017665,1351653108364017665,1.611094e+12,2021-01-19,-0500,,@MartinSLewis Hi I purchased a faulty XBox X S...,en,[],...,,,,,,[],,,,


In [10]:
# Deploy data_crawlers function on stocks list
for i in stocks: 
    data_crawlers(i)
    df = twint.storage.panda.Tweets_df
    stock_df = stock_df.append(df)

In [11]:
# Read tweets as strings
stock_df['tweet'] = stock_df['tweet'].astype(str)
stock_df.head()

id      conversation_id    created_at  \
microsoft 0  1351680836580601857  1351600853459857410  1.611101e+12   
          1  1351680824828166144  1351680824828166144  1.611101e+12   
          2  1351680820927467521  1351680820927467521  1.611101e+12   
          3  1351680820252033024  1351680820252033024  1.611101e+12   
          4  1351680814942195713  1351680814942195713  1.611101e+12   

                            date timezone place  \
microsoft 0  2021-01-19 18:59:59    -0500         
          1  2021-01-19 18:59:56    -0500         
          2  2021-01-19 18:59:55    -0500         
          3  2021-01-19 18:59:55    -0500         
          4  2021-01-19 18:59:53    -0500         

                                                         tweet language  \
microsoft 0  @grimshaw_leon @Treyarch @Microsoft I’m not un...       en   
          1  I won 4 achievements in Microsoft Sudoku (UWP)...       en   
          2  I won the Flipping Out achievement in Microsof...       en   
          3  Be it @Microsoft or @Google CEO but...  फिर भी...       hi   
          4  I won the Mummy’s Boy achievement in Microsoft...       en   

                       hashtags cashtags  ...  geo source user_rt_id user_rt  \
microsoft 0                  []       []  ...                                  
          1  [trueachievements]       []  ...                                  
          2  [trueachievements]       []  ...                                  
          3          [indvsaus]       []  ...                                  
          4  [trueachievements]       []  ...                                  

             retweet_id                                           reply_to  \
microsoft 0              [{'screen_name': 'grimshaw_leon', 'name': 'leo...   
          1                                                             []   
          2                                                             []   
          3                                                             []   
          4                                                             []   

            retweet_date translate trans_src  trans_dest  
microsoft 0                                               
          1                                               
          2                                               
          3                                               
          4                                               

[5 rows x 38 columns]

In [12]:
# Reformat Date
stock_df['date'] = [x[:10] for x in stock_df['date']]
# Reset Index
stock_df.reset_index(inplace=True)
# Drop Superfluous Column
stock_df = stock_df.drop(columns='level_1')
stock_df = stock_df.rename(columns={'level_0':'Stock'})

stock_df.head()

,Stock,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,microsoft,1351680836580601857,1351600853459857410,1.611101e+12,2021-01-19,-0500,,@grimshaw_leon @Treyarch @Microsoft I’m not un...,en,[],...,,,,,,"[{'screen_name': 'grimshaw_leon', 'name': 'leo...",,,,
1,microsoft,1351680824828166144,1351680824828166144,1.611101e+12,2021-01-19,-0500,,I won 4 achievements in Microsoft Sudoku (UWP)...,en,[trueachievements],...,,,,,,[],,,,
2,microsoft,1351680820927467521,1351680820927467521,1.611101e+12,2021-01-19,-0500,,I won the Flipping Out achievement in Microsof...,en,[trueachievements],...,,,,,,[],,,,
3,microsoft,1351680820252033024,1351680820252033024,1.611101e+12,2021-01-19,-0500,,Be it @Microsoft or @Google CEO but... फिर भी...,hi,[indvsaus],...,,,,,,[],,,,
4,microsoft,1351680814942195713,1351680814942195713,1.611101e+12,2021-01-19,-0500,,I won the Mummy’s Boy achievement in Microsoft...,en,[trueachievements],...,,,,,,[],,,,


In [13]:
# Reset the Index to be both Date and Stock
stock_df = stock_df.set_index(['date','Stock'])
stock_df.tail()

id      conversation_id    created_at  \
date       Stock                                                             
2021-01-19 moderna  1351656068888350728  1351652172824510467  1.611095e+12   
           moderna  1351655978643705860  1351652453813514246  1.611095e+12   
           moderna  1351655975669915648  1351344878031142920  1.611095e+12   
           moderna  1351655952945184775  1351655952945184775  1.611095e+12   
           moderna  1351655936931344390  1351655936931344390  1.611095e+12   

                   timezone place  \
date       Stock                    
2021-01-19 moderna    -0500         
           moderna    -0500         
           moderna    -0500         
           moderna    -0500         
           moderna    -0500         

                                                                tweet  \
date       Stock                                                        
2021-01-19 moderna  @nytimes Currently and Pfizer and moderna fact...   
           moderna  @Manuelfilosofia 1) Estoy de acuerdo en que ha...   
           moderna                   @star_schmoney Moderna 1st dose.   
           moderna  Vacunación: arrancó la segunda etapa y el país...   
           moderna  📻#Ahora | @rosaliafucello en #Animate "Buscamo...   

                   language                     hashtags cashtags  \
date       Stock                                                    
2021-01-19 moderna       en                           []       []   
           moderna       es                           []       []   
           moderna       de                           []       []   
           moderna       es                           []       []   
           moderna       es  [ahora, animate, sanisidro]       []   

                                user_id  ... geo source user_rt_id  user_rt  \
date       Stock                         ...                                  
2021-01-19 moderna            129971221  ...                                  
           moderna  1272571735838265345  ...                                  
           moderna   937661853626486784  ...                                  
           moderna            186584578  ...                                  
           moderna   967587399034396674  ...                                  

                   retweet_id  \
date       Stock                
2021-01-19 moderna              
           moderna              
           moderna              
           moderna              
           moderna              

                                                             reply_to  \
date       Stock                                                        
2021-01-19 moderna  [{'screen_name': 'nytimes', 'name': 'The New Y...   
           moderna  [{'screen_name': 'Manuelfilosofia', 'name': 'M...   
           moderna  [{'screen_name': 'star_schmoney', 'name': 'Sta...   
           moderna                                                 []   
           moderna                                                 []   

                   retweet_date translate  trans_src trans_dest  
date       Stock                                                 
2021-01-19 moderna                                               
           moderna                                               
           moderna                                               
           moderna                                               
           moderna                                               

[5 rows x 37 columns]

In [14]:
# Filter just on the relevant columns
stock_df = stock_df.iloc[:, : 7]

In [15]:
# Create a Master copy of the stock_df
master_df = stock_df.copy()
master_df = master_df.reset_index(drop=True)

# Perform Sentiment Analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer 
analyzer = SentimentIntensityAnalyzer()
tweet_sent = {'Compound':[],
                 'Negative':[],
                 'Neutral':[],
                 'Positive':[]}
# Get sentiment from tweets
for index, row in master_df.iterrows():
    try:
        tweet_sentiment = analyzer.polarity_scores(row["tweet"])
        tweet_sent["Compound"].append(tweet_sentiment["compound"])
        tweet_sent["Negative"].append(tweet_sentiment["neg"])
        tweet_sent["Neutral"].append(tweet_sentiment["neu"])
        tweet_sent["Positive"].append(tweet_sentiment["pos"])
    except AttributeError:
        pass
# Add the newly created scores to the previously created dataframe
tweet_sent_df = pd.DataFrame(tweet_sent)
master_df = tweet_sent_df.join(master_df)
master_df.head()

,Compound,Negative,Neutral,Positive,id,conversation_id,created_at,timezone,place,tweet,language
0,0.0000,0.0,1.000,0.000,1351680836580601857,1351600853459857410,1.611101e+12,-0500,,@grimshaw_leon @Treyarch @Microsoft I’m not un...,en
1,0.5719,0.0,0.748,0.252,1351680824828166144,1351680824828166144,1.611101e+12,-0500,,I won 4 achievements in Microsoft Sudoku (UWP)...,en
2,0.5719,0.0,0.802,0.198,1351680820927467521,1351680820927467521,1.611101e+12,-0500,,I won the Flipping Out achievement in Microsof...,en
3,0.0000,0.0,1.000,0.000,1351680820252033024,1351680820252033024,1.611101e+12,-0500,,Be it @Microsoft or @Google CEO but... फिर भी...,hi
4,0.5719,0.0,0.802,0.198,1351680814942195713,1351680814942195713,1.611101e+12,-0500,,I won the Mummy’s Boy achievement in Microsoft...,en


In [16]:
# Create a dataframe list and another DF
dataframes = [x for x in stocks]
df = stock_df.reset_index()
df.head()

,date,Stock,id,conversation_id,created_at,timezone,place,tweet,language
0,2021-01-19,microsoft,1351680836580601857,1351600853459857410,1.611101e+12,-0500,,@grimshaw_leon @Treyarch @Microsoft I’m not un...,en
1,2021-01-19,microsoft,1351680824828166144,1351680824828166144,1.611101e+12,-0500,,I won 4 achievements in Microsoft Sudoku (UWP)...,en
2,2021-01-19,microsoft,1351680820927467521,1351680820927467521,1.611101e+12,-0500,,I won the Flipping Out achievement in Microsof...,en
3,2021-01-19,microsoft,1351680820252033024,1351680820252033024,1.611101e+12,-0500,,Be it @Microsoft or @Google CEO but... फिर भी...,hi
4,2021-01-19,microsoft,1351680814942195713,1351680814942195713,1.611101e+12,-0500,,I won the Mummy’s Boy achievement in Microsoft...,en


In [17]:
# Create New Dataframes
for x in dataframes:
    vars()[x] = df[df['Stock'] == x]
    vars()[x] = vars()[x].reset_index(drop=True)
for x in dataframes:
    display(vars()[x])

,date,Stock,id,conversation_id,created_at,timezone,place,tweet,language
0,2021-01-19,microsoft,1351680836580601857,1351600853459857410,1.611101e+12,-0500,,@grimshaw_leon @Treyarch @Microsoft I’m not un...,en
1,2021-01-19,microsoft,1351680824828166144,1351680824828166144,1.611101e+12,-0500,,I won 4 achievements in Microsoft Sudoku (UWP)...,en
2,2021-01-19,microsoft,1351680820927467521,1351680820927467521,1.611101e+12,-0500,,I won the Flipping Out achievement in Microsof...,en
3,2021-01-19,microsoft,1351680820252033024,1351680820252033024,1.611101e+12,-0500,,Be it @Microsoft or @Google CEO but... फिर भी...,hi
4,2021-01-19,microsoft,1351680814942195713,1351680814942195713,1.611101e+12,-0500,,I won the Mummy’s Boy achievement in Microsoft...,en
...,...,...,...,...,...,...,...,...,...
1090,2021-01-19,microsoft,1351653168980107271,1351653168980107271,1.611094e+12,-0500,,"Cruise, GM partner with Microsoft to commercia...",en
1091,2021-01-19,microsoft,1351653160817975296,1351653160817975296,1.611094e+12,-0500,,eu to rindo muito com a história da vez que a ...,pt
1092,2021-01-19,microsoft,1351653138953068544,1351651325788913664,1.611094e+12,-0500,,@enertopiacorp @GM @Microsoft As usual nothing...,en
1093,2021-01-19,microsoft,1351653108364017665,1351653108364017665,1.611094e+12,-0500,,@MartinSLewis Hi I purchased a faulty XBox X S...,en


,date,Stock,id,conversation_id,created_at,timezone,place,tweet,language
0,2021-01-19,sony,1351680702899773445,1351680702899773445,1.611101e+12,-0500,,Sony San Diego trabaja en un juego de acción q...,es
1,2021-01-19,sony,1351680686529236993,1351680686529236993,1.611101e+12,-0500,,バイクはホンダ、車はトヨタ、家電はSONY、服はユニクロ。 庶民として生きてきたのである、こ...,ja
2,2021-01-19,sony,1351680633504890882,1351680633504890882,1.611101e+12,-0500,,Harry O bankrolled Death Row when they were ba...,en
3,2021-01-19,sony,1351680620645199879,1351564220803018757,1.611101e+12,-0500,,@Hamzelll @brittyfun ok sony,en
4,2021-01-19,sony,1351680589628203008,1351680589628203008,1.611101e+12,-0500,,That #PS5 custom faceplate seller that Sony th...,en
...,...,...,...,...,...,...,...,...,...
995,2021-01-19,sony,1351657082399961097,1351599007022383106,1.611095e+12,-0500,,@HaLo_Larrie28 Beside that the manager is not ...,en
996,2021-01-19,sony,1351657045888479237,1351560453776879618,1.611095e+12,-0500,,@elflaco7ven @PlayStation Very disappointing o...,en
997,2021-01-19,sony,1351657011205832709,1351657011205832709,1.611095e+12,-0500,,🚨MAS SPIN-OFFS🚨 The Illuminerdi reporta que So...,es
998,2021-01-19,sony,1351656978586734595,1351656978586734595,1.611095e+12,-0500,,@PlayStation are you guys building these PS5's...,en


,date,Stock,id,conversation_id,created_at,timezone,place,tweet,language
0,2021-01-19,pfizer,1351680822756184075,1351586077837062148,1.611101e+12,-0500,,@PhilTBH See this is why I want Pfizer and not...,en
1,2021-01-19,pfizer,1351680798550843393,1351680798550843393,1.611101e+12,-0500,,بعد تطعيم الجرعه الثانيه pfizer - biontech ا...,ar
2,2021-01-19,pfizer,1351680794616606723,1351551080430264325,1.611101e+12,-0500,,@norabar @msalnacion @tunykollmann Averiguaron...,es
3,2021-01-19,pfizer,1351680790149652480,1351680790149652480,1.611101e+12,-0500,,La vacuna de Pfizer protege desde la primera d...,ca
4,2021-01-19,pfizer,1351680782214057984,1351677076013772802,1.611101e+12,-0500,,@todonoticias Asi y todo es mas segura que la ...,es
...,...,...,...,...,...,...,...,...,...
995,2021-01-19,pfizer,1351669969612902403,1351605511104450566,1.611098e+12,-0500,,@WelshGovernment @vaughangething No point in h...,en
996,2021-01-19,pfizer,1351669965645107205,1351669965645107205,1.611098e+12,-0500,,Primero que Panamá sería uno de los primeros p...,es
997,2021-01-19,pfizer,1351669962730049537,1351617288642961408,1.611098e+12,-0500,,@kristynwongtam Yes I’m sure the Premier wants...,en
998,2021-01-19,pfizer,1351669946242265090,1351669946242265090,1.611098e+12,-0500,,É um absurdo o que eles estão fazendo com os i...,pt


,date,Stock,id,conversation_id,created_at,timezone,place,tweet,language
0,2021-01-19,regeneron,1351673433487142912,1351673433487142912,1.611099e+12,-0500,,I'm so sorry. Request the regeneron infusion f...,en
1,2021-01-19,regeneron,1351672972143075328,1351672822330871808,1.611099e+12,-0500,,"$REGN +61,494 01/19/2021 $ARKK bought 61,494 ...",en
2,2021-01-19,regeneron,1351672793147068418,1349769359816863747,1.611099e+12,-0500,,@itchdoctor @CellCellPress @WUSTLmed @WUDeptMe...,en
3,2021-01-19,regeneron,1351671798627237892,1351665042232659968,1.611099e+12,-0500,,"@han_uhh_ @ttstime Sounds horrible, sending po...",en
4,2021-01-19,regeneron,1351671569991532546,1351671569991532546,1.611099e+12,-0500,,The Democrats are whining about Josh Hawley? W...,en
...,...,...,...,...,...,...,...,...,...
1091,2021-01-13,regeneron,1349396931152404480,1349396931152404480,1.610556e+12,-0500,,"@doctorgaona La FDA USA,autoriza de emergencia...",es
1092,2021-01-13,regeneron,1349396535268831240,1349396535268831240,1.610556e+12,-0500,,#Top #Healthcare #Stocks ☆Pfizer Inc ☆PTC T...,ca
1093,2021-01-13,regeneron,1349396056426115074,1349386097051258881,1.610556e+12,-0500,,@vincentglad il faut arreter d associer hcq et...,fr
1094,2021-01-13,regeneron,1349395585762267136,1349385356173520896,1.610556e+12,-0500,,@The_Answer3_ @FISchmidtLab Ich baue auf @FISc...,de


,date,Stock,id,conversation_id,created_at,timezone,place,tweet,language
0,2021-01-19,moderna,1351680810907279360,1351607772132085760,1.611101e+12,-0500,,@fordnation ALL LARGE OUTBREAK COMMUNITIES ACR...,en
1,2021-01-19,moderna,1351680773993205762,1351680773993205762,1.611101e+12,-0500,,Public health authorities in Calif are seeking...,en
2,2021-01-19,moderna,1351680765189386243,1351630020771536896,1.611101e+12,-0500,,@fordnation ALL LARGE OUTBREAK COMMUNITIES ACR...,en
3,2021-01-19,moderna,1351680718775017472,1351668010419974144,1.611101e+12,-0500,,@JenLingeman @Laurie_Garrett @moderna_tx GMAB ...,en
4,2021-01-19,moderna,1351680714576637953,1351674848377008129,1.611101e+12,-0500,,@fordnation ALL LARGE OUTBREAK COMMUNITIES ACR...,en
...,...,...,...,...,...,...,...,...,...
995,2021-01-19,moderna,1351656068888350728,1351652172824510467,1.611095e+12,-0500,,@nytimes Currently and Pfizer and moderna fact...,en
996,2021-01-19,moderna,1351655978643705860,1351652453813514246,1.611095e+12,-0500,,@Manuelfilosofia 1) Estoy de acuerdo en que ha...,es
997,2021-01-19,moderna,1351655975669915648,1351344878031142920,1.611095e+12,-0500,,@star_schmoney Moderna 1st dose.,de
998,2021-01-19,moderna,1351655952945184775,1351655952945184775,1.611095e+12,-0500,,Vacunación: arrancó la segunda etapa y el país...,es


In [18]:
# Sentiment Analysis on the Individual Stocks
for x in dataframes:
    tweet_sent = {'Compound':[],
                 'Negative':[],
                 'Neutral':[],
                 'Positive':[]}
    # Get sentiment from tweets
    for index, row in vars()[x].iterrows():
        try:
            tweet_sentiment = analyzer.polarity_scores(row["tweet"])
            tweet_sent["Compound"].append(tweet_sentiment["compound"])
            tweet_sent["Negative"].append(tweet_sentiment["neg"])
            tweet_sent["Neutral"].append(tweet_sentiment["neu"])
            tweet_sent["Positive"].append(tweet_sentiment["pos"])
        except AttributeError:
            pass
    # Add the newly created scores to the previously created dataframe
    tweet_sent_df = pd.DataFrame(tweet_sent)
    vars()[x] = tweet_sent_df.join(vars()[x])
    

In [99]:
# Display the sentiment analysis
final_list = []
for x in dataframes:
    #final_df = [display(vars()[x])]
    each_list = [vars()[x]]
    final_list += each_list
first_df = pd.DataFrame(final_list[0])
second_df = pd.DataFrame(final_list[1])
third_df = pd.DataFrame(final_list[2])
fourth_df = pd.DataFrame(final_list[3])
fifth_df = pd.DataFrame(final_list[4])
final_df = pd.concat([first_df,second_df,third_df,fourth_df,fifth_df])
final_df

,Compound,Negative,Neutral,Positive,date,Stock,id,conversation_id,created_at,timezone,place,tweet,language
0,0.0000,0.000,1.000,0.000,2021-01-19,microsoft,1351680836580601857,1351600853459857410,1.611101e+12,-0500,,@grimshaw_leon @Treyarch @Microsoft I’m not un...,en
1,0.5719,0.000,0.748,0.252,2021-01-19,microsoft,1351680824828166144,1351680824828166144,1.611101e+12,-0500,,I won 4 achievements in Microsoft Sudoku (UWP)...,en
2,0.5719,0.000,0.802,0.198,2021-01-19,microsoft,1351680820927467521,1351680820927467521,1.611101e+12,-0500,,I won the Flipping Out achievement in Microsof...,en
3,0.0000,0.000,1.000,0.000,2021-01-19,microsoft,1351680820252033024,1351680820252033024,1.611101e+12,-0500,,Be it @Microsoft or @Google CEO but... फिर भी...,hi
4,0.5719,0.000,0.802,0.198,2021-01-19,microsoft,1351680814942195713,1351680814942195713,1.611101e+12,-0500,,I won the Mummy’s Boy achievement in Microsoft...,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0000,0.000,1.000,0.000,2021-01-19,moderna,1351656068888350728,1351652172824510467,1.611095e+12,-0500,,@nytimes Currently and Pfizer and moderna fact...,en
996,-0.7430,0.142,0.858,0.000,2021-01-19,moderna,1351655978643705860,1351652453813514246,1.611095e+12,-0500,,@Manuelfilosofia 1) Estoy de acuerdo en que ha...,es
997,0.0000,0.000,1.000,0.000,2021-01-19,moderna,1351655975669915648,1351344878031142920,1.611095e+12,-0500,,@star_schmoney Moderna 1st dose.,de
998,-0.2960,0.061,0.939,0.000,2021-01-19,moderna,1351655952945184775,1351655952945184775,1.611095e+12,-0500,,Vacunación: arrancó la segunda etapa y el país...,es


In [100]:
data = final_df[['Compound','Negative','Positive','Stock']]
data

,Compound,Negative,Positive,Stock
0,0.0000,0.000,0.000,microsoft
1,0.5719,0.000,0.252,microsoft
2,0.5719,0.000,0.198,microsoft
3,0.0000,0.000,0.000,microsoft
4,0.5719,0.000,0.198,microsoft
...,...,...,...,...
995,0.0000,0.000,0.000,moderna
996,-0.7430,0.142,0.000,moderna
997,0.0000,0.000,0.000,moderna
998,-0.2960,0.061,0.000,moderna


In [104]:
data['Sentiment_Score'] = np.nan
data.loc[data['Compound'] >= 0.05, 'Sentiment_Score'] = 1
data.loc[data['Compound'] <= - 0.05, 'Sentiment_Score'] = -1
data.loc[(data['Compound'] < 0.05) & (data['Compound'] > -0.05), 'Sentiment_Score']= 0
data.head()

C:\Users\julia\Anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\julia\Anaconda3\envs\nlp\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\julia\Anaconda3\envs\nlp\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

,Compound,Negative,Positive,Stock,Sentiment_Score
0,0.0000,0.0,0.000,microsoft,0.0
1,0.5719,0.0,0.252,microsoft,1.0
2,0.5719,0.0,0.198,microsoft,1.0
3,0.0000,0.0,0.000,microsoft,0.0
4,0.5719,0.0,0.198,microsoft,1.0


In [106]:
# Convert [-1, 0, 1] to [negative, neutral, and positive]

# Negative
data.loc[data['Sentiment_Score'] == -1, 'Sentiment_Score'] = 'Negative'

# Neutral

data.loc[data['Sentiment_Score'] == 0, 'Sentiment_Score'] = 'Neutral'

# Postive
data.loc[data['Sentiment_Score'] == 1, 'Sentiment_Score'] = 'Positive'

# View Into DataFrame
data.head()

C:\Users\julia\Anaconda3\envs\nlp\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Compound,Negative,Positive,Stock,Sentiment_Score
0,0.0000,0.0,0.000,microsoft,Neutral
1,0.5719,0.0,0.252,microsoft,Positive
2,0.5719,0.0,0.198,microsoft,Positive
3,0.0000,0.0,0.000,microsoft,Neutral
4,0.5719,0.0,0.198,microsoft,Positive


In [110]:
# Ensemble Learning

In [131]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [132]:
# Create our features
X = data.drop(columns='Sentiment_Score')
X = pd.get_dummies(X)

# Create our target
target = ["Sentiment_Score"]
y = data.loc[:, target].copy()

In [133]:
X.describe() 

,Compound,Negative,Positive,Stock_microsoft,Stock_moderna,Stock_pfizer,Stock_regeneron,Stock_sony
count,5191.000000,5191.000000,5191.000000,5191.000000,5191.000000,5191.000000,5191.000000,5191.000000
mean,0.040617,0.043346,0.060553,0.210942,0.192641,0.192641,0.211135,0.192641
std,0.376411,0.079818,0.101493,0.408017,0.394412,0.394412,0.408153,0.394412
min,-0.967300,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.241100,0.069000,0.099000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.974400,0.707000,0.819000,1.000000,1.000000,1.000000,1.000000,1.000000


In [134]:
y['Sentiment_Score'].value_counts()

Neutral     2453
Positive    1515
Negative    1223
Name: Sentiment_Score, dtype: int64

In [135]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [136]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [137]:
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)

In [138]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [139]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train_scaled, y_train)

C:\Users\julia\Anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


BalancedRandomForestClassifier(random_state=1)

In [140]:
# Calculated the balanced accuracy score
y_pred = brf.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9994717379820391

In [141]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[307,   0,   0],
       [  0, 630,   1],
       [  0,   0, 360]], dtype=int64)

In [142]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

   Negative       1.00      1.00      1.00      1.00      1.00      1.00       307
    Neutral       1.00      1.00      1.00      1.00      1.00      1.00       631
   Positive       1.00      1.00      1.00      1.00      1.00      1.00       360

avg / total       1.00      1.00      1.00      1.00      1.00      1.00      1298

